<a href="https://colab.research.google.com/github/chouhandiksha/bigdataproject/blob/colab/notebooks/diksha/analyse_NY_mobility_poverty_imptime_series_2020.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Spark SQL Documentation:** 
https://spark.apache.org/docs/2.2.0/sql-programming-guide.html

In [1]:
# add time information at the end of every cell
!pip install ipython-autotime
%load_ext autotime

time: 1.71 ms (started: 2021-04-20 04:42:18 +00:00)


In [2]:
# Install required dependancies
!pip install pyspark
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

     |████████████████████████████████| 212.3MB 71kB/s 
     |████████████████████████████████| 204kB 42.5MB/s 
  Created wheel for pyspark: filename=pyspark-3.1.1-py2.py3-none-any.whl size=212767604 sha256=f27e9956e9df5f9b67f11ac22da61c6ca8a6843015beb5f17e7d924a6b2dc8aa
  Stored in directory: /root/.cache/pip/wheels/0b/90/c0/01de724414ef122bd05f056541fb6a0ecf47c7ca655f8b3c0f
Successfully built pyspark
The following additional packages will be installed:
  openjdk-8-jre-headless
Suggested packages:
  openjdk-8-demo openjdk-8-source libnss-mdns fonts-dejavu-extra
  fonts-ipafont-gothic fonts-ipafont-mincho fonts-wqy-microhei
  fonts-wqy-zenhei fonts-indic
The following NEW packages will be installed:
  openjdk-8-jdk-headless openjdk-8-jre-headless
0 upgraded, 2 newly installed, 0 to remove and 31 not upgraded.
Need to get 36.5 MB of archives.
After this operation, 143 MB of additional disk space will be used.
Selecting previously unselected package openjdk-8-jre-headless:amd64.
(Reading

In [3]:
# Import modules
import numpy as np
import pandas as pd
import datetime
import matplotlib.pyplot as plt
from pathlib import Path
%matplotlib inline

import pyspark
from pyspark.sql import *
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf

time: 76.1 ms (started: 2021-04-20 04:43:08 +00:00)


In [4]:
import altair as alt
alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

time: 196 ms (started: 2021-04-20 04:43:08 +00:00)


In [5]:
# create the session
conf = SparkConf().set("spark.ui.port", "4050")

# create the context
sc = pyspark.SparkContext(conf=conf)
spark = SparkSession.builder.getOrCreate()

time: 4.13 s (started: 2021-04-20 04:43:08 +00:00)


In [6]:
#sc.stop()

time: 4.26 ms (started: 2021-04-20 04:43:12 +00:00)


In [7]:
# Mount drive with data
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
time: 31.1 s (started: 2021-04-20 04:43:12 +00:00)


In [8]:
# Set path to data folder
path = Path('drive/MyDrive/big-data-project/data/clean-data')
city = 'ny'

time: 4.44 ms (started: 2021-04-20 04:43:44 +00:00)


In [9]:
# Read data into dataframe
df_soc = spark.read.format('csv').option('header','true').option('quote',"\"").option('escape',"\"").load(str(path/city/'social/2020/*.csv'))
df_soc.show()

+---+------------+--------------------+--------------------+------------+---------------------------+--------------------------+------------------------------------------+----------------------------+----------------------+------------------------+--------------------+-------------------------------+-------------------------------+--------------------+-------------------------+--------------------------+----------------------+----------------------------+---------------------------+-----------------------------+
|_c0|         cbg|    date_range_start|      date_range_end|device_count|distance_traveled_from_home|bucketed_distance_traveled|median_dwell_at_bucketed_distance_traveled|completely_home_device_count|median_home_dwell_time|bucketed_home_dwell_time|at_home_by_each_hour|part_time_work_behavior_devices|full_time_work_behavior_devices|    destination_cbgs|delivery_behavior_devices|median_non_home_dwell_time|candidate_device_count|bucketed_away_from_home_time|median_percentage_time_

In [10]:
df_soc.createOrReplaceTempView('clean_ny')
df_soc = spark.sql('SELECT cbg, date_range_start as date, device_count, completely_home_device_count, part_time_work_behavior_devices, full_time_work_behavior_devices FROM clean_ny WHERE device_count > 5')
df_soc.show()

+------------+--------------------+------------+----------------------------+-------------------------------+-------------------------------+
|         cbg|                date|device_count|completely_home_device_count|part_time_work_behavior_devices|full_time_work_behavior_devices|
+------------+--------------------+------------+----------------------------+-------------------------------+-------------------------------+
|360470064002|2020-02-03T00:00:...|          64|                          18|                              7|                              5|
|360810384001|2020-02-03T00:00:...|          84|                          21|                             12|                              8|
|360850170103|2020-02-03T00:00:...|         250|                          68|                             41|                             26|
|360050213021|2020-02-03T00:00:...|          92|                          26|                             11|                              6|
|36005

In [11]:
df_soc = df_soc.withColumn("date", df_soc['date'].substr(0, 10))

time: 46.4 ms (started: 2021-04-20 04:44:04 +00:00)


In [12]:
df_soc.show()

+------------+----------+------------+----------------------------+-------------------------------+-------------------------------+
|         cbg|      date|device_count|completely_home_device_count|part_time_work_behavior_devices|full_time_work_behavior_devices|
+------------+----------+------------+----------------------------+-------------------------------+-------------------------------+
|360470064002|2020-02-03|          64|                          18|                              7|                              5|
|360810384001|2020-02-03|          84|                          21|                             12|                              8|
|360850170103|2020-02-03|         250|                          68|                             41|                             26|
|360050213021|2020-02-03|          92|                          26|                             11|                              6|
|360050253004|2020-02-03|          52|                          13|         

In [13]:
# Add completely home percentage column
df_soc = df_soc.withColumn('completely_home_percentage', (df_soc['completely_home_device_count']/df_soc['device_count']) * 100)
df_soc = df_soc.withColumn('part_time_work_percentage', (df_soc['part_time_work_behavior_devices']/df_soc['device_count']) * 100)
df_soc = df_soc.withColumn('full_time_work_percentage', (df_soc['full_time_work_behavior_devices']/df_soc['device_count']) * 100)

#df_soc.show()

time: 123 ms (started: 2021-04-20 04:44:05 +00:00)


In [14]:
df_soc.printSchema()

root
 |-- cbg: string (nullable = true)
 |-- date: string (nullable = true)
 |-- device_count: string (nullable = true)
 |-- completely_home_device_count: string (nullable = true)
 |-- part_time_work_behavior_devices: string (nullable = true)
 |-- full_time_work_behavior_devices: string (nullable = true)
 |-- completely_home_percentage: double (nullable = true)
 |-- part_time_work_percentage: double (nullable = true)
 |-- full_time_work_percentage: double (nullable = true)

time: 56.2 ms (started: 2021-04-20 04:44:05 +00:00)


In [15]:
#Create temp view
df_soc.createOrReplaceTempView('mobility')

time: 36.9 ms (started: 2021-04-20 04:44:05 +00:00)




---


demographic data






---



In [16]:
# Read poverty data
# Read data into RDD
df_demographic = spark.read.format('csv').option('header','true').option('quote',"\"").option('escape',"\"").load(str(path/city/'ny.csv'))
df_demographic.createOrReplaceTempView('demographic')
df_demographic.show()

+------------+---------+------------------+------------------+
|         cbg|pop_total|poverty_percentage|    perc_whiteonly|
+------------+---------+------------------+------------------+
|360050001000|        0|               0.0|               0.0|
|360050001001|     7503|               0.0|12.435025989604158|
|360050002000|        0|               0.0|               0.0|
|360050002001|     2114|16.130558183538316|35.856196783349105|
|360050002002|     2168|18.911439114391143| 48.06273062730627|
|360050002003|      969| 8.152734778121776|49.742002063983485|
|360050004000|        0|               0.0|               0.0|
|360050004001|      646|32.972136222910216| 44.27244582043344|
|360050004002|     1400|25.571428571428573| 75.35714285714286|
|360050004003|     3393| 1.569905213270142| 33.65753020925435|
|360050004004|      541|15.196998123827393| 20.33271719038817|
|360050016001|     3427|  25.1531952144733|38.400933761307265|
|360050016002|      662| 62.99093655589124| 73.56495468

In [17]:
df_mob_demo = spark.sql('SELECT mobility.*, demographic.poverty_percentage from mobility INNER JOIN demographic ON mobility.cbg = demographic.cbg')
df_mob_demo.createOrReplaceTempView('demographic_mobility')
df_mob_demo.show()


+------------+----------+------------+----------------------------+-------------------------------+-------------------------------+--------------------------+-------------------------+-------------------------+------------------+
|         cbg|      date|device_count|completely_home_device_count|part_time_work_behavior_devices|full_time_work_behavior_devices|completely_home_percentage|part_time_work_percentage|full_time_work_percentage|poverty_percentage|
+------------+----------+------------+----------------------------+-------------------------------+-------------------------------+--------------------------+-------------------------+-------------------------+------------------+
|360470064002|2020-02-03|          64|                          18|                              7|                              5|                    28.125|                  10.9375|                   7.8125| 19.22705314009662|
|360810384001|2020-02-03|          84|                          21|             

In [18]:
# DIVIDING THE PEOPLE TOP 20 PERCENT AND BOTTOM 20 PERCENT

df_mob_demo_high_class =  spark.sql('SELECT * FROM demographic_mobility WHERE poverty_percentage < 20')
grouped_df_mob_demo_high_class = df_mob_demo_high_class.groupBy("date").mean("completely_home_percentage").withColumnRenamed('avg(completely_home_percentage)','completely_home_percentage')
grouped_df_mob_demo_high_class.createOrReplaceTempView('mob_demo_high_class')


df_mob_demo_low_class =  spark.sql('SELECT * FROM demographic_mobility WHERE poverty_percentage > 80')
grouped_df_mob_demo_low_class = df_mob_demo_low_class.groupBy("date").mean("completely_home_percentage").withColumnRenamed('avg(completely_home_percentage)','completely_home_percentage')
grouped_df_mob_demo_low_class.createOrReplaceTempView('mob_demo_low_class')

grouped_df_mob_demo_combined = spark.sql("""

SELECT h.date,h.completely_home_percentage as high_comp_home_perc,
l.completely_home_percentage as low_comp_home_perc  
FROM mob_demo_high_class h INNER JOIN mob_demo_low_class l ON h.date = l.date

""")


time: 272 ms (started: 2021-04-20 04:44:07 +00:00)


---
Visualization 

---

In [19]:
df = grouped_df_mob_demo_combined.toPandas()
df

,date,high_comp_home_perc,low_comp_home_perc
0,2020-02-26,20.948572,20.489095
1,2020-04-13,57.527185,47.815293
2,2020-06-24,39.998245,40.032680
3,2020-06-08,42.439567,31.688312
4,2020-09-12,33.992552,15.840081
...,...,...,...
361,2020-04-05,54.866796,40.241935
362,2020-05-01,53.816496,52.280702
363,2020-10-25,36.063622,28.735632
364,2020-12-28,34.252238,34.923077


time: 4min (started: 2021-04-20 04:44:07 +00:00)


In [20]:
df_str = df

time: 1.1 ms (started: 2021-04-20 04:48:08 +00:00)


In [21]:
base = alt.Chart(df_str.reset_index()).encode(x='date'
)

alt.layer(
    base.mark_line(color='green').encode(y='high_comp_home_perc'),
    base.mark_line(color='red').encode(y='low_comp_home_perc')

)

alt.LayerChart(...)

time: 61.1 ms (started: 2021-04-20 04:48:08 +00:00)


Filtering based on important dates

March 8, 2020: 
NYC issues guidelines to avoid densely packed buses, subways, or trains.

March 12, 2020:
Broadway shuts down.

March 14, 2020:
First two COVID-19 deaths in NYS.

March 22, 2020:
NYS on Pause Program begins, all non-essential workers must stay home.

March 28, 2020:
Governor Cuomo halts all nonessential construction sites in NYS.











In [22]:
#Conversion of datetime format
df['date'] = pd.to_datetime(df['date'],utc= True)
df['date'] = [datetime.datetime.date(d) for d in df['date']] 

time: 10.2 ms (started: 2021-04-20 04:48:08 +00:00)


In [23]:
df1 = df[(df['date']>datetime.date(2020,3,5)) & (df['date']<datetime.date(2020,3,31))]  
df1['date'] = df1['date'].astype(str)

time: 18 ms (started: 2021-04-20 04:48:08 +00:00)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [24]:
base = alt.Chart(df1.reset_index()).encode(x='date'
).properties(width = 1000)
alt.layer(
    base.mark_line(color='green').encode(y='high_comp_home_perc'),
    base.mark_line(color='red').encode(y='low_comp_home_perc')
).interactive()

alt.LayerChart(...)

time: 29.4 ms (started: 2021-04-20 04:48:08 +00:00)


April 10, 2020:
NYS records more COVID19 cases than any other country 





In [25]:
df2 = df[(df['date']>datetime.date(2020,4,2)) & (df['date']<datetime.date(2020,4,24))]  
df2['date'] = df2['date'].astype(str)

time: 4.69 ms (started: 2021-04-20 04:48:08 +00:00)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [26]:
base = alt.Chart(df2.reset_index()).encode(x='date'
).properties(width = 1000)
alt.layer(
    base.mark_line(color='green').encode(y='high_comp_home_perc'),
    base.mark_line(color='red').encode(y='low_comp_home_perc')
).interactive()

alt.LayerChart(...)

time: 34.4 ms (started: 2021-04-20 04:48:08 +00:00)


**July 6, 2020: NYC begins Phase 3 of reopening, without indoor dining**

--- 

In [27]:
df3 = df[(df['date']>datetime.date(2020,7,1)) & (df['date']<datetime.date(2020,7,16))]  
df3['date'] = df3['date'].astype(str)

time: 4.99 ms (started: 2021-04-20 04:48:08 +00:00)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [28]:
base = alt.Chart(df3.reset_index()).encode(x='date'
).properties(width = 1000)
alt.layer(
    base.mark_line(color='green').encode(y='high_comp_home_perc'),
    base.mark_line(color='red').encode(y='low_comp_home_perc')
).interactive()

alt.LayerChart(...)

time: 33.1 ms (started: 2021-04-20 04:48:08 +00:00)


October 28, 2020:
New York tops a half-million confirmed COVID-19 cases

---

In [29]:
df4 = df[(df['date']>datetime.date(2020,10,20)) & (df['date']<datetime.date(2020,11,8))]  
df4['date'] = df4['date'].astype(str)

time: 5.67 ms (started: 2021-04-20 04:48:08 +00:00)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [30]:
base = alt.Chart(df4.reset_index()).encode(x='date'
).properties(width = 1000)
alt.layer(
    base.mark_line(color='green').encode(y='high_comp_home_perc'),
    base.mark_line(color='red').encode(y='low_comp_home_perc')
).interactive()

alt.LayerChart(...)

time: 34.9 ms (started: 2021-04-20 04:48:08 +00:00)
